In [40]:
from google.colab import files
files.upload()  # Select the kaggle.json file you downloaded


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sudhanshugusain","key":"adaff061ce557b03ad66b20e2628da5b"}'}

In [41]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [42]:
!kaggle datasets download -d sooryaprakash12/cleaned-indian-recipes-dataset


  0% 0.00/3.22M [00:00<?, ?B/s]
100% 3.22M/3.22M [00:00<00:00, 133MB/s]


In [43]:
!unzip cleaned-indian-recipes-dataset.zip -d recipes

Archive:  cleaned-indian-recipes-dataset.zip
  inflating: recipes/Cleaned_Indian_Food_Dataset.csv  


### 1. IMPORTING LIBRARIES AND DEPENDANCIES ALONG WITH DATA

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/content/recipes/Cleaned_Indian_Food_Dataset.csv")

df.head(1)

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10


In [71]:
df.shape

(5938, 9)

In [72]:
df.isnull().sum()

TranslatedRecipeName      0
TranslatedIngredients     0
TotalTimeInMins           0
Cuisine                   0
TranslatedInstructions    0
URL                       0
Cleaned-Ingredients       0
image-url                 0
Ingredient-count          0
dtype: int64

In [73]:
df.duplicated().sum()

0

### 2. DROPPING IRRELEVANT COLUMNS

In [74]:
df.columns

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins',
       'Cuisine', 'TranslatedInstructions', 'URL', 'Cleaned-Ingredients',
       'image-url', 'Ingredient-count'],
      dtype='object')

In [75]:
columns_to_drop = ["TotalTimeInMins","Cleaned-Ingredients","Ingredient-count","Cuisine"]

In [76]:
new_df = df.drop(columns=columns_to_drop)
new_df.head(1)

,TranslatedRecipeName,TranslatedIngredients,TranslatedInstructions,URL,image-url
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,https://www.archanaskitchen.com/images/archana...


In [77]:
# Renaming COlumns
new_df.rename(columns={
    "TranslatedRecipeName":"title",
    "TranslatedIngredients":"ingridients",
    "TranslatedInstructions":"directions",
    "URL":"link"
} , inplace = True)

In [78]:
new_df.head(2)

,title,ingridients,directions,link,image-url
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,https://www.archanaskitchen.com/images/archana...
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...","To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,https://www.archanaskitchen.com/images/archana...


### 3. EXPLORING DIRECTIONS COLUMN

In [79]:
new_df.iloc[0]['directions']

'To begin making the Masala Karela Recipe,de-seed the karela and slice.\nDo not remove the skin as the skin has all the nutrients.\nAdd the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles.\nRelease the pressure immediately and open the lids.\nKeep aside.Heat oil in a heavy bottomed pan or a kadhai.\nAdd cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan.\nStir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again.\nCover the pan and simmer Masala Karela stirring occasionally until everything comes together well.\nTurn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.\n'

In [80]:
type(new_df.iloc[0]['directions'])

str

In [81]:
def split_sentences_and_clean(input_string):
    """
    Function to split sentences at \n, remove full stops, punctuations, and empty spaces at the end of sentences, and convert them into a list.

    Parameters:
        input_string (str): The input string containing sentences separated by \n.

    Returns:
        list: A list containing the cleaned sentences from the input string.
    """
    # Split the input string into sentences at \n
    sentences = input_string.split('\n')

    # Clean each sentence by removing full stops, punctuations, and empty spaces at the end
    cleaned_sentences = [sentence.rstrip('.!? ').rstrip() for sentence in sentences]

    # Return the list of cleaned sentences
    return cleaned_sentences

In [82]:
new_df['directions'] = new_df['directions'].apply(split_sentences_and_clean)

In [83]:
type(new_df.iloc[0]['directions'])

list

In [84]:
new_df.iloc[0]['directions']

['To begin making the Masala Karela Recipe,de-seed the karela and slice',
 'Do not remove the skin as the skin has all the nutrients',
 'Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles',
 'Release the pressure immediately and open the lids',
 'Keep aside.Heat oil in a heavy bottomed pan or a kadhai',
 'Add cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan',
 'Stir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again',
 'Cover the pan and simmer Masala Karela stirring occasionally until everything comes together well',
 'Turn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family',
 '']

### 4. EXPLORING INGRIDIENTS COLUMN

In [85]:
new_df.columns

Index(['title', 'ingridients', 'directions', 'link', 'image-url'], dtype='object')

In [86]:
new_df.iloc[0]['ingridients']

'1 tablespoon Red Chilli powder,3 tablespoon Gram flour (besan),2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),2 teaspoons Turmeric powder (Haldi),Salt - to taste,1 tablespoon Amchur (Dry Mango Powder),6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Sunflower Oil - as required,1 Onion - thinly sliced'

In [87]:
type(new_df.iloc[0]['ingridients'])

str

In [88]:
new_df['ingridients'] = new_df['ingridients'].apply(split_sentences_and_clean)

In [89]:
type(new_df.iloc[0]['ingridients'])

list

In [90]:
new_df.iloc[0]['ingridients']

['1 tablespoon Red Chilli powder,3 tablespoon Gram flour (besan),2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),2 teaspoons Turmeric powder (Haldi),Salt - to taste,1 tablespoon Amchur (Dry Mango Powder),6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Sunflower Oil - as required,1 Onion - thinly sliced']

### 5. MAKING A NEW COLUMN BY COMBINING DIRECTIONS AND INGRIDIENTS

In [91]:
new_df["tags"] = new_df["ingridients"]+new_df["directions"]

### 6. MAKING A NEW DATATSET OF ONLY IMP COLUMNS

In [92]:
new_df.columns

Index(['title', 'ingridients', 'directions', 'link', 'image-url', 'tags'], dtype='object')

In [93]:
recipe = new_df[['title','link','image-url','tags']]

### 7. SOME NLP( Natural Language Processing ) PROCESSING

In [94]:
# COnverting list to str
recipe['tags'] = recipe['tags'].apply(lambda x: " ".join(x))

<ipython-input-94-5e8fd47a2fcf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe['tags'] = recipe['tags'].apply(lambda x: " ".join(x))


In [95]:
# Converting all to lowercase
recipe['tags'] = recipe['tags'].apply(lambda x:x.lower())

<ipython-input-95-9cd5e1436db3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe['tags'] = recipe['tags'].apply(lambda x:x.lower())


In [96]:
recipe.head(2)

,title,link,image-url,tags
0,Masala Karela Recipe,https://www.archanaskitchen.com/masala-karela-...,https://www.archanaskitchen.com/images/archana...,"1 tablespoon red chilli powder,3 tablespoon gr..."
1,Spicy Tomato Rice (Recipe),https://www.archanaskitchen.com/spicy-tomato-r...,https://www.archanaskitchen.com/images/archana...,"2 teaspoon cashew - or peanuts, 1/2 teaspoon ..."


In [97]:
# IMPORTING NLP LIBRARIES and ML libraries
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [98]:
# STEMMING OF WORDS
ps = PorterStemmer()
def stems(text):
    T = []
    for i in text.split():
        T.append(ps.stem(i))

    return " ".join(T)


recipe['tags'] = recipe['tags'].apply(stems)

<ipython-input-98-961605adb501>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe['tags'] = recipe['tags'].apply(stems)


In [99]:
# Converting words to vector using CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(recipe['tags']).toarray()

In [100]:
vector.shape

(5938, 5000)

In [101]:
similarity = cosine_similarity(vector)
similarity.shape

(5938, 5938)

### 8. PREDICTION FUNCTION

In [110]:
def recommend(recipes):
    index = recipe[recipe['title'] == recipes].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(recipe.iloc[i[0]].title)

In [111]:
new_df.iloc[0]['title']

'Masala Karela Recipe'

In [112]:
# Predict test
recommend("Masala Karela Recipe")

Bitter gourd spice vegetable recipe
Besan vegetable recipe
Crispy Vegetable Recipe - Crispy Bhujia
Sindhi Karela Bhaji Recipe - Sindhi Style Bitter Gourd Sabzi
Bihari Bitter Gourd Potato Vegetable Recipe


### 9. SAVING

In [115]:
with open('/content/artifacts/recipes.pkl', 'wb') as file:
    pickle.dump(recipe, file)

# Save similarity to a file named 'similarity.pkl'
with open('/content/artifacts/similarity.pkl', 'wb') as file:
    pickle.dump(similarity, file)

In [116]:
# Mounting Google drive for saving on local PC

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [117]:
import shutil

# Assuming 'file_path' is the path of your file in Colab
file_path = '/content/artifacts/similarity.pkl'

# Specify the destination path on Google Drive
drive_path = '/content/drive/My Drive/similarity.pkl'

# Copy the file to Google Drive
shutil.copyfile(file_path, drive_path)

'/content/drive/My Drive/similarity.pkl'